In [2]:
from util.backtest import *
from util.factor_util import *
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools
from tqdm import tqdm
import asyncio
from tardis_client import TardisClient, Channel
from binance.client import Client


In [2]:
url = 'https://api.tardis.dev/v1/exchanges/binance-futures'

result = requests.get(url).json()
result.keys()

dict_keys(['id', 'name', 'enabled', 'availableSince', 'availableChannels', 'availableSymbols', 'datasets', 'incidentReports'])

In [3]:
usdt_symbols = [sym['id'] for sym in result['availableSymbols'] if 'usdt' in sym['id'] and sym['type']=='perpetual']
print(usdt_symbols[:5])
len(usdt_symbols)


['btcusdt', 'ethusdt', 'bchusdt', 'xrpusdt', 'eosusdt']


175

In [3]:
dt = pd.date_range(start='2023/2/1', end='2023/2/2',freq = 'D')

In [3]:
tardis_client = TardisClient(api_key="TD.bGGkN0U-Q23J11Fg.BM25zW2D02TrP1J.QPwJos8qqNCSsqF.TX4WVlWhyd6WOOQ.ISFXAVw3BXatkSZ.1tLt")

messages = tardis_client.replay(
exchange="binance-futures",
from_date="2023-01-01",
to_date="2023-01-02",
filters=[Channel(name="topLongShortAccountRatio", symbols=["sandusdt"])]
)

In [ ]:
async for local_timestamp, message in messages:
    print(local_timestamp,int(local_timestamp.timestamp()*1000) + 8*3600 *1000,message)


In [8]:
data = message['data']
data['receive_timestamp'] = int(local_timestamp.timestamp()*1000)
data

{'symbol': 'AXSUSDT',
 'longAccount': '0.6678',
 'longShortRatio': '2.0102',
 'shortAccount': '0.3322',
 'timestamp': 1672617000000,
 'receive_timestamp': 1672588755713}

In [85]:

filters = [
    {"channel": "topLongShortAccountRatio", "symbols": ["btcusdt"]},
]
qs_params = {"from": "2022-02-01", "to": "2022-02-02", "offset": 3, "filters": json.dumps(filters)}

headers = {"Authorization": "TD.bGGkN0U-Q23J11Fg.BM25zW2D02TrP1J.QPwJos8qqNCSsqF.TX4WVlWhyd6WOOQ.ISFXAVw3BXatkSZ.1tLt"}

url = "https://api.tardis.dev/v1/data-feeds/binance-futures"

response = requests.get(url, headers=headers, params=qs_params, stream=True)


parts = response.text.split(" ")
local_timestamp = parts[0]
message = json.loads(parts[1])


In [86]:

for line in response.iter_lines():
    # empty lines in response are being used as markers
    # for disconnect events that occurred when collecting the data
    if len(line) <= 1:
        continue

    parts = line.decode("utf-8").split(" ")
    local_timestamp = parts[0]
    message = json.loads(parts[1])
    # local_timestamp string marks message arrival timestamp
    # message is a message dict as provided by exchange real-time stream
    print(local_timestamp, message)


2022-02-01T00:03:15.1491960Z {'stream': 'btcusdt@topLongShortAccountRatio', 'generated': True, 'data': {'symbol': 'BTCUSDT', 'longAccount': '0.4541', 'longShortRatio': '0.8318', 'shortAccount': '0.5459', 'timestamp': 1643673300000}}
